In [1]:
from knnor import data_augment
import numpy as np
import math
import random
from copy import deepcopy

In [2]:



def predict_classification(X,y,new_vector, num_neighbors_to_test,expected_class_index):
    '''
    this function is used to validate
    whether new point generated is close to
    same label points
    '''
    from sklearn.neighbors import KNeighborsClassifier
    posit=np.argsort(abs((X-new_vector)*(X-new_vector)).sum(axis=1))
    classes = y[posit[0:num_neighbors_to_test]]
    return np.sum(classes==expected_class_index)==classes.shape[0]

def check_duplicates( new_row,old_rows):
    '''
    check if the new row
    is already preent in the old rows
    '''
    for row in old_rows:
        same=True
        for i in range(len(row)):
            if new_row[i]!=row[i]:
                same=False
                continue
        if same:
            return True                            
    return False

def get_minority_label_index(X,y):
    '''
    find the minority label
    and the indices at which minority label
    is present
    '''
    # find the minority label
    uniq_labels=np.unique(y)
    # count for each label
    dic_nry={}

    for uniq_label in uniq_labels:
        dic_nry[uniq_label]=0

    for y_val in y:
        dic_nry[y_val]+=1

    # then which one is the minority label?
    minority_label=-1
    minimum_count=np.inf
    for k,v in dic_nry.items():
        if minimum_count>v:
            minimum_count=v
            minority_label=k


    # now get the indices of the minority labels
    minority_indices=[]
    for i in range(y.shape[0]):
        if y[i]==minority_label:
            minority_indices.append(i)

    return minority_label,minority_indices

def good_count_neighbors(X,y):
    '''
    find the good number of neighbors to use
    this function is used on auto pilot
    '''
    minority_label,minority_indices=get_minority_label_index(X,y)
    X_minority=X[minority_indices]
    y_minority=y[minority_indices]
    count_greater=y_minority.shape[0]
    for i in range(X_minority.shape[0]):
        this_point_features=X_minority[i]
        dist = ((X_minority-this_point_features)*(X_minority-this_point_features)).sum(axis=1)
        mean_dist=np.mean(dist)
#         print(dist,mean_dist)
        this_point_count_lesser = (dist < mean_dist).sum()
        count_greater=min(this_point_count_lesser,count_greater)        
    return count_greater





# following function
# to get the savitzky golay filter
# https://en.wikipedia.org/wiki/Savitzky%E2%80%93Golay_filter
# https://scipy.github.io/old-wiki/pages/Cookbook/SavitzkyGolay
# https://stackoverflow.com/questions/20618804/how-to-smooth-a-curve-in-the-right-way

def savitzky_golay(y, window_size, order, deriv=0, rate=1):         
    import numpy as np
    from math import factorial

    try:
        window_size = np.abs(int(window_size))
        order = np.abs(int(order))
    except ValueError:
        raise ValueError("window_size and order have to be of type int")
    if window_size % 2 != 1 or window_size < 1:
        raise TypeError("window_size size must be a positive odd number")
    if window_size < order + 2:
        raise TypeError("window_size is too small for the polynomials order")
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
    # pad the signal at the extremes with
    # values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve( m[::-1], y, mode='valid')


def check_enough_minorities(X,y,num_neighbors):
    '''
    ideally, the total number of minority points should be
    1 more than the total number of neighbors    
    '''
    minority_label,minority_indices=get_minority_label_index(X,y)
    if len(minority_indices)<=num_neighbors:
        print("You want to use ",num_neighbors,"neighbors, but minority data size = ",len(minority_indices))
        return False
    return True


def calculate_count_to_add(X,y,final_proportion):
    '''
    Calculate the number of artificial points to be generated so that
    (count_minority_existing+count_artificial_minority)/count_majority_existing=final_proportion
    '''
#     minority_label,minority_indices=get_minority_label_index(X,y)
#     majority_indices=[]
#     for i in range(0,y.shape[0]):
#         if i not in minority_indices:
#             majority_indices.append(i)
#     count_minority=len(minority_indices)
#     count_majority=len(majority_indices)
#     new_minority=int((final_proportion*count_majority)-count_minority)
#     if new_minority<1:
#         return -1
    
    
    # extra code
    count_to_add=int(final_proportion*len(X))
    return count_to_add
    
#     return new_minority







In [3]:

# def knnor_over_sample(X,y,n_to_sample,num_neighbors,proportion,max_dist_point,intra=True):
def fit_resample(X,y,**params):
    threshold_cannot_use=10

    # check for number of neighbors
    if 'num_neighbors' in params.keys():
        num_neighbors=params['num_neighbors']
    else:
        good_neighbor_count=good_count_neighbors(X,y)
        if good_neighbor_count<=1:
            print("Too few neighbors")
            return X,y
        num_neighbors=random.randrange(1,good_neighbor_count)


    if 'max_dist_point' in params.keys():
        max_dist_point=params['max_dist_point']
    else:
        max_dist_point=max_threshold_dist(X,y,num_neighbors)

    if 'proportion_minority' in params.keys():
        '''
        proportion of minority population to use
        '''
        proportion_minority=params['proportion_minority']
        inter=False
    else:
        proportion_intra=calculate_distance_threshold(X,y,num_neighbors,intra=False)
        proportion_minority=proportion_intra
        inter=True



#     if not check_enough_minorities(X,y,num_neighbors):
#         print("Too few minorities")
#         return X,y

    if 'final_proportion' in params.keys():
        '''
        final minority pop = what percentage of majority pop
        '''
        final_proportion=params['final_proportion']

    else:
        final_proportion=1


    n_to_sample=calculate_count_to_add(X,y,final_proportion)

    original_n_neighbors=num_neighbors
    original_max_dist_point=max_dist_point    
    original_proportion=proportion_minority

    minority_label,minority_indices=get_minority_label_index(X,y)
    X_minority=X[minority_indices]
    y_minority=y[minority_indices]
    majority_indices=[]
    for i in range(0,y.shape[0]):
        if i not in minority_indices:
            majority_indices.append(i)
    print(len(majority_indices),len(minority_indices),y.shape)
    X_majority=X[majority_indices]
    y_majority=y[majority_indices]

    if not inter:
        internal_distance = np.linalg.norm(X_minority - X_minority[:,None], axis = -1)
        internal_distance = np.sort(internal_distance)
        knd=internal_distance[:,num_neighbors]        
        knd_sorted = np.sort(knd)        
    else:
        external_distance=np.linalg.norm(X_majority - X_minority[:,None], axis = -1)
        external_distance = np.sort(external_distance)
        knd=external_distance[:,num_neighbors]   
        knd_sorted=-np.sort(-knd)

    threshold_dist = knd_sorted[math.floor(proportion_minority*len(knd_sorted))]

    X_new_minority=[]
    N = n_to_sample
    consecutive_cannot_use=0
    while N>0:
        for i in range(X_minority.shape[0]):
            if inter:
                if knd[i]>threshold_dist:
                    continue
            else:
                if knd[i]<threshold_dist:
                    continue
            if N==0:
                break
            v = X_minority[i,:]
            val=np.sort( abs((X_minority-v)*(X_minority-v)).sum(axis=1) )
            # sort neighbors by distance
            # obviously will have to ignore the 
            # first term as its a distance to iteself
            # which wil be 0
            posit=np.argsort(abs((X_minority-v)*(X_minority-v)).sum(axis=1))
            kv = X_minority[posit[1:num_neighbors+1],:]
            alphak = random.uniform(0,max_dist_point)
            m0 = v
            for j in range(num_neighbors):
                m1 = m0 + alphak * (kv[j,:] - m0)
                m0 = m1
            num_neighbors_to_test=math.floor(math.sqrt(num_neighbors))
            can_use=predict_classification(X,y,m0, num_neighbors_to_test,minority_label)
            can_use=can_use and not(check_duplicates(m0,X_minority))
            can_use=can_use and not(check_duplicates(m0,X_new_minority))                            
            if can_use:
                consecutive_cannot_use=0
                num_neighbors=min(num_neighbors+1,original_n_neighbors)
                max_dist_point=min(max_dist_point+0.01,original_max_dist_point)
                proportion_minority=max(proportion_minority-0.01,original_proportion)
                threshold_dist = knd_sorted[math.floor(proportion_minority*len(knd_sorted))]                
                X_new_minority.append(m0)
                N-=1
            else:
                consecutive_cannot_use+=1
                if consecutive_cannot_use>=threshold_cannot_use:
                    num_neighbors=max(num_neighbors-1,2)
                    max_dist_point=max(max_dist_point-0.01,0.01)
                    proportion_minority=min(proportion_minority+0.01,0.9)
                    threshold_dist = knd_sorted[math.floor(proportion_minority*len(knd_sorted))]
                    consecutive_cannot_use=0

    y_new_minority=[minority_label for i in range(len(X_new_minority))]        
    X_new_minority=np.array(X_new_minority)
    X_new_all=np.concatenate((X, X_new_minority), axis=0)
    y_new_all=np.concatenate((y, y_new_minority), axis=0)

    return X_new_all, y_new_all, X_new_minority, y_new_minority







In [4]:

def calculate_distance_threshold(X,y,num_neighbors,intra=True):
    '''
    returns the distance threshold, based on the intra parameter
    if intra is chosen, returns the cut-off point for distances to
    kth nearest neighbor of same class
    in inter is chosen, returns the cut-off point for distances to 
    kth nearest neighbor of opposite class

    '''
    win_size=5 #positive odd number
    pol_order=2
    alpha=0.0001 # low value for denominator 0 case
    minortiy_label=1
    minority_indices=list(range(0,len(X)))
#     minority_label,minority_indices=get_minority_label_index(X,y)
    X_minority=X[minority_indices]
    y_minority=y[minority_indices]
    

    if intra:
        internal_distance = np.linalg.norm(X_minority - X_minority[:,None], axis = -1)
        internal_distance = np.sort(internal_distance)
        knd=internal_distance[:,num_neighbors]

        knd_sorted = np.sort(knd)


   


    # normalize it        
    normalized_dist= (knd_sorted-np.min(knd_sorted))/(np.max(knd_sorted)-np.min(knd_sorted)+alpha)

    # apply golay        
    normalized_dist = savitzky_golay(normalized_dist, win_size, pol_order) # window size 51, polynomial order 3
#     plt.plot(normalized_dist)
#     plt.title("NOrmalized distance intra"+str(intra))
#     plt.show()
    normalized_dist=np.diff(normalized_dist)

    sin_values=np.abs(np.sin(np.arctan(normalized_dist)))
#     plt.title("Sin differential - to get maxima intra"+str(intra))
#     plt.plot(sin_values)
#     plt.show()
    first_maxima_index=np.argmax(sin_values)
#     print("Maxima is at ",first_maxima_index)
    proportion=first_maxima_index/sin_values.shape[0]
    return proportion



# following function to calculate maximum
# threshold distance
# while placing a point
def max_threshold_dist(X,y,num_neighbors):
    '''
    This function calculates the maximum distance between any two points in the minority class
    It also calculates the minimum distance between a point in the minority and a point
    in the majority class
    the value returned is the minimum of the two
    '''
    minority_label,minority_indices=get_minority_label_index(X,y)
    X_minority=X[minority_indices]
    y_minority=y[minority_indices]
    majority_indices=[]
    for i in range(0,y.shape[0]):
        if i not in minority_indices:
            majority_indices.append(i)
    print(len(majority_indices),len(minority_indices),y.shape)
    X_majority=X[majority_indices]
    y_majority=y[majority_indices]



    # calculate inter distance
    internal_distance = np.linalg.norm(X_minority - X_minority[:,None], axis = -1)
    internal_distance=internal_distance.flatten()
    max_internal_distance=np.max(internal_distance)
    
    min_internal_distance=np.min(internal_distance[internal_distance>0])    



    # additional code change
    max_allowed_distance=min_internal_distance/max_internal_distance
    
    return max_allowed_distance




In [5]:

model_nm="VanillaVAE"
X_vals_arr=np.load("logs/"+model_nm+"/enc/test_enc.npy")

In [6]:
X=deepcopy(X_vals_arr)
y=np.array([0 for i in range(len(X))])

In [7]:
how_many=500
random_indices = np.random.choice(len(X), size=how_many, replace=False)

X = X[random_indices, :]
y=np.array([0 for i in range(len(X))])

In [8]:
print("X=",X.shape,"y=",y.shape)
print("Original Data:")
print("************************************")


X= (500, 256) y= (500,)
Original Data:
************************************


In [21]:



final_proportion=0.5
num_neighbors=10
n_to_sample=calculate_count_to_add(X,y,final_proportion)
print("Number of new points=",n_to_sample)
max_dist_point=max_threshold_dist(X,y,num_neighbors)
print("max_dist_point",max_dist_point)
proportion_intra=calculate_distance_threshold(X,y,num_neighbors,intra=True)
proportion_minority=proportion_intra
print("Proportion of population used = ",proportion_minority)




X_minority=X
y_minority=y



internal_distance = np.linalg.norm(X_minority - X_minority[:,None], axis = -1)
internal_distance = np.sort(internal_distance)
knd=internal_distance[:,num_neighbors]        
knd_sorted = np.sort(knd)        


threshold_dist = knd_sorted[math.floor(proportion_minority*len(knd_sorted))]
print("Threshold distance is ",threshold_dist)

Number of new points= 250
0 500 (500,)
max_dist_point 0.18143772
Proportion of population used =  0.9879759519038076
Threshold distance is  6.6030464


In [22]:
threshold_cannot_use=10
original_n_neighbors=num_neighbors
original_max_dist_point=max_dist_point
original_proportion=proportion_minority
X_new_minority=[]
N = n_to_sample
consecutive_cannot_use=0
while N>0:
    for i in range(X_minority.shape[0]):

        if knd[i]<threshold_dist:
            continue
        if N==0:
            break
        v = X_minority[i,:]
        val=np.sort( abs((X_minority-v)*(X_minority-v)).sum(axis=1) )
        # sort neighbors by distance
        # obviously will have to ignore the 
        # first term as its a distance to iteself
        # which wil be 0
        posit=np.argsort(abs((X_minority-v)*(X_minority-v)).sum(axis=1))
        kv = X_minority[posit[1:num_neighbors+1],:]
        alphak = random.uniform(0,max_dist_point)
        m0 = v
#         print(m0)
        for j in range(num_neighbors):
#             print(kv[j,:] ,"-", m0)
#             print(m0,"+",alphak,"*", (kv[j,:] - m0))
            m1 = m0 + alphak * (kv[j,:] - m0)
            m0 = m1
#             print("res",m0)
        num_neighbors_to_test=math.floor(math.sqrt(num_neighbors))
        can_use= not(check_duplicates(m0,X_minority))
        can_use=can_use and not(check_duplicates(m0,X_new_minority))                            
        if can_use:
            consecutive_cannot_use=0
            num_neighbors=min(num_neighbors+1,original_n_neighbors)
            max_dist_point=min(max_dist_point+0.01,original_max_dist_point)
            proportion_minority=max(proportion_minority-0.01,original_proportion)
            threshold_dist = knd_sorted[math.floor(proportion_minority*len(knd_sorted))]                
#             print(m0)
#             print("*"*10)
            X_new_minority.append(m0)
            N-=1
        else:
            consecutive_cannot_use+=1
            if consecutive_cannot_use>=threshold_cannot_use:
                num_neighbors=max(num_neighbors-1,2)
                max_dist_point=max(max_dist_point-0.01,0.01)
                proportion_minority=min(proportion_minority+0.01,0.9)
                threshold_dist = knd_sorted[math.floor(proportion_minority*len(knd_sorted))]
                consecutive_cannot_use=0


In [23]:
X_new_minority=np.array(X_new_minority)

In [24]:
X_new_minority.shape

(250, 256)

In [25]:
n_to_sample

250

In [26]:
np.save("logs/"+model_nm+"/enc/test_aug_enc.npy",X_new_minority)